In [1]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from transformers import ViTFeatureExtractor, ViTModel, ViTConfig, DistilBertModel, DistilBertConfig
from tqdm.notebook import tqdm
from torch.autograd import Variable
from datetime import datetime,timedelta
import time
from data_preparation import fun

In [2]:
MODEL_PATH = './data/'

In [3]:
class PressureEncorder(nn.Module):
    def __init__(self, image_size = 41, patch_size = 4, num_channels = 40, encoder_stride = 4):
        super(PressureEncorder, self).__init__()
        config = ViTConfig(image_size = 41, patch_size = 4, num_channels = 10, encoder_stride = 4)
        self.hidden_size = int((image_size // encoder_stride)**2 + 1) * config.hidden_size
        self.ViT = ViTModel(config)
        
        self.linear = nn.Linear(self.hidden_size + 20, 10)
        
    def forward(self, x):
        pressure, surge, time = x
        time = time.float()
        hidden = self.ViT(pressure).last_hidden_state.reshape(-1, self.hidden_size)
        x = torch.concat([hidden, surge, time], dim = 1)
        x = self.linear(x)
        return x

In [4]:
# num classes : numer of output parameters
class LSTMSurge(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, more_hidden_size=10, final_values=10):
        super(LSTMSurge, self).__init__()
        
        self.num_classes = num_classes
        self.num_layers = num_layers
        self.input_size = input_size
        self.hidden_size = hidden_size
        
        self.lstm = nn.LSTM(
            input_size=input_size, 
            hidden_size=hidden_size,
            num_layers=num_layers, 
            batch_first=True
        )

        layers = [
            nn.Linear(hidden_size, more_hidden_size), 
            nn.GELU(), 
            nn.Linear(more_hidden_size, final_values)
        ]
        self.mlp = nn.Sequential(*layers)

    def forward(self, x):
        h_0 = Variable(
            torch.zeros(self.num_layers, x.size(0), self.hidden_size)
        )
        
        c_0 = Variable(
            torch.zeros(self.num_layers, x.size(0), self.hidden_size)
        )
        
        ula, (h_out, _) = self.lstm(x, (h_0, c_0))
        
        h_out = h_out.view(-1, self.hidden_size)
        
        out = self.mlp(h_out)
        
        return out

In [63]:
w = torch.linspace(1, 0.1, 10)[np.newaxis]

def custom_weighted_losses(output, target):
    loss = torch.mean(w * (output - target)**2)
    return loss

## Data Loading

Here the data is loaded and scaled

In [6]:
X_train = np.load('./data/X_train_surge.npz')
Y_train = pd.read_csv('./data/Y_train_surge.csv')
X_test = np.load('./data/X_test_surge.npz')

# train
slp_train = X_train['slp']
t_slp_train = X_train['t_slp']

t_surge1_input_train = X_train['t_surge1_input']
t_surge2_input_train = X_train['t_surge2_input']

surge1_input_train = X_train['surge1_input']
surge2_input_train = X_train['surge2_input']

mean_surge1_input_train = np.mean(surge1_input_train, axis=1)
std_surge1_input_train = np.std(surge1_input_train, axis=1)
mean_surge2_input_train = np.mean(surge2_input_train, axis=1)
std_surge2_input_train = np.std(surge2_input_train, axis=1)

scaled_surge1_input_train = (surge1_input_train - mean_surge1_input_train[:,None]) / std_surge1_input_train[:,None]
scaled_surge2_input_train = (surge2_input_train - mean_surge2_input_train[:,None]) / std_surge2_input_train[:,None]

t_surge1_output_train = X_train['t_surge1_output']
t_surge2_output_train = X_train['t_surge2_output']

# test
slp_test = X_test['slp']
t_slp_test = X_test['t_slp']

t_surge1_input_test = X_test['t_surge1_input']
t_surge2_input_test = X_test['t_surge2_input']

surge1_input_test = X_test['surge1_input']
surge2_input_test = X_test['surge2_input']

mean_surge1_input_test = np.mean(surge1_input_test, axis=1)
std_surge1_input_test = np.std(surge1_input_test, axis=1)
mean_surge2_input_test = np.mean(surge2_input_test, axis=1)
std_surge2_input_test = np.std(surge2_input_test, axis=1)

scaled_surge1_input_test = (surge1_input_test - mean_surge1_input_test[:,None]) / std_surge1_input_test[:,None]
scaled_surge2_input_test = (surge2_input_test - mean_surge2_input_test[:,None]) / std_surge2_input_test[:,None]

t_surge1_output_test = X_test['t_surge1_output']
t_surge2_output_test = X_test['t_surge2_output']

Now we need to divide the output

In [7]:
Y_1 = Y_train[['surge1_t0', 'surge1_t1', 'surge1_t2', 'surge1_t3', 'surge1_t4', 'surge1_t5', 'surge1_t6', 'surge1_t7', 'surge1_t8', 'surge1_t9']].to_numpy()
Y_2 = Y_train[['surge2_t0', 'surge2_t1', 'surge2_t2', 'surge2_t3', 'surge2_t4', 'surge2_t5', 'surge2_t6', 'surge2_t7', 'surge2_t8', 'surge2_t9']].to_numpy()

Code to have a series of pressures the same as the surges

In [8]:
def find_nearest(array, value):
    array = np.asarray(array)
    idx = (np.abs(array - value)).argmin()
    return idx

In [9]:
pressures_same_time_1 = np.empty((*(t_surge1_input_train.shape), 41, 41))
for i, time_series in enumerate(t_surge1_input_train):
    for j, time in enumerate(time_series):
        idx = find_nearest(t_slp_train[i,:].flatten(), time)
        pressures_same_time_1[i, j, :, :] = slp_train[i, idx, :, :]

pressures_same_time_2 = np.empty((*(t_surge2_input_train.shape), 41, 41))
for i, time_series in enumerate(t_surge2_input_train):
    for j, time in enumerate(time_series):
        idx = find_nearest(t_slp_train[i,:].flatten(), time)
        pressures_same_time_2[i, j, :, :] = slp_train[i, idx, :, :]

In [10]:
mean_pressures_same_time_1 = np.mean(pressures_same_time_1, axis=(1,2,3))
std_pressures_same_time_1 = np.std(pressures_same_time_1, axis=(1,2,3))
mean_pressures_same_time_2 = np.mean(pressures_same_time_2, axis=(1,2,3))
std_pressures_same_time_2 = np.std(pressures_same_time_2, axis=(1,2,3))

scaled_pressures_same_time_1 = (pressures_same_time_1 - mean_pressures_same_time_1[:,None, None, None]) / std_pressures_same_time_1[:,None, None, None]
scaled_pressures_same_time_2 = (pressures_same_time_2 - mean_pressures_same_time_2[:,None, None, None]) / std_pressures_same_time_2[:,None, None, None]


In [11]:
def hour_rounder(t):
    return (t.replace(second=0, microsecond=0, minute=0, hour=t.hour) + timedelta(hours=t.minute//30))

In [12]:
def time_to_hour(array):
    hours_array = np.empty_like(array)
    for i, times in enumerate(array):
        for j, t in enumerate(times):
            if t<0:
                tt = (datetime(1970,1,1) + timedelta(seconds=int(t))).timetuple()
            else:
                tt = (datetime.fromtimestamp(int(t))).timetuple() 
            hours_array[i][j] = tt.tm_yday * 24 + tt.tm_hour
    return hours_array / (366 * 24)

In [21]:
hours_in_year_surge_1_train = time_to_hour(t_surge1_input_train)
hours_in_year_surge_2_train = time_to_hour(t_surge2_input_train)
hours_in_year_surge_1_test = time_to_hour(t_surge1_input_test)
hours_in_year_surge_2_test = time_to_hour(t_surge1_input_test)
hours_in_year_surge_1_output_train = time_to_hour(t_surge1_output_train)
hours_in_year_surge_2_output_train = time_to_hour(t_surge2_output_train)
hours_in_year_slp_train = time_to_hour(t_slp_train)
hours_in_year_slp_test = time_to_hour(t_slp_test)

In [22]:
datalen = len(surge1_input_train)
trainlen = int(0.9 * datalen)
vallen = datalen - trainlen
train_idx, val_idx = torch.utils.data.random_split(np.arange(datalen), [trainlen, vallen])

pressure1_train, pressure1_val = pressures_same_time_1[train_idx], pressures_same_time_1[val_idx]
surge1_train, surge1_val = surge1_input_train[train_idx], surge1_input_train[val_idx]
t_surge1_train, t_surge1_val = hours_in_year_surge_1_train[train_idx], hours_in_year_surge_1_train[val_idx]
Y_1_train, Y_1_val = Y_1[train_idx], Y_1[val_idx]

train_data = list(zip(pressure1_train, surge1_train, t_surge1_train, Y_1_train))
val_data = list(zip(pressure1_val, surge1_val, t_surge1_val, Y_1_val))

batch_size = 8

train_dataloader1 = DataLoader(
    train_data,
    batch_size=batch_size,
    shuffle=True
)

val_dataloader1 = DataLoader(
    val_data,
    batch_size=batch_size,
    shuffle=False
)

In [39]:
datalen = len(surge2_input_train)
trainlen = int(0.9 * datalen)
vallen = datalen - trainlen
train_idx, val_idx = torch.utils.data.random_split(np.arange(datalen), [trainlen, vallen])

pressure2_train, pressure2_val = pressures_same_time_2[train_idx], pressures_same_time_2[val_idx]
surge2_train, surge2_val = surge2_input_train[train_idx], surge2_input_train[val_idx]
t_surge2_train, t_surge2_val = hours_in_year_surge_2_train[train_idx], hours_in_year_surge_2_train[val_idx]
Y_2_train, Y_2_val = Y_2[train_idx], Y_2[val_idx]

train_data = list(zip(pressure2_train, surge2_train, t_surge2_train, Y_2_train))
val_data = list(zip(pressure2_val, surge2_val, t_surge2_val, Y_2_val))

batch_size = 8

train_dataloader2 = DataLoader(
    train_data,
    batch_size=batch_size,
    shuffle=True
)

val_dataloader2 = DataLoader(
    val_data,
    batch_size=batch_size,
    shuffle=False
)

## Data Training

Training of the transformer to compute the features from the pressure

In [23]:
model = PressureEncorder()
device = torch.device('cuda')
model = model.to(device)
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

In [40]:
model2 = PressureEncorder()
device = torch.device('cuda')
model2 = model2.to(device)
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model2.parameters(), lr=1e-4)

In [24]:
epochs = 10

for epoch in range(epochs):
    model.train()
    for x1, x2, x3, y in tqdm(train_dataloader1, total = len(train_dataloader1), leave=False):
        x1, x2, x3, y = x1.to(device), x2.to(device), x3.to(device), y.to(device)
        x1 = x1.type(torch.cuda.FloatTensor)
        x2 = x2.type(torch.cuda.FloatTensor)
        x3 = x3.type(torch.cuda.FloatTensor)
        y = y.type(torch.cuda.FloatTensor)
        optimizer.zero_grad()
        pred = model((x1, x2, x3))
        loss = criterion(pred, y)
        loss.backward()
        optimizer.step()
    model.eval()
    val_loss = 0
    with torch.no_grad():
        for x1, x2, x3, y in tqdm(val_dataloader1, total = len(val_dataloader1), leave = False):
            x1, x2, x3, y = x1.to(device), x2.to(device), x3.to(device), y.to(device)
            x1 = x1.type(torch.cuda.FloatTensor)
            x2 = x2.type(torch.cuda.FloatTensor)
            x3 = x3.type(torch.cuda.FloatTensor)
            y = y.type(torch.cuda.FloatTensor)
            pred = model((x1, x2, x3))
            loss = criterion(pred, y)
            val_loss += loss.item()
    val_loss /= (len(val_dataloader1)*batch_size)
    print(f'Epoch {epoch+1}: Validation Loss = {val_loss}')

  0%|          | 0/630 [00:00<?, ?it/s]

  0%|          | 0/70 [00:00<?, ?it/s]

Epoch 1: Validation Loss = 0.1288423269454922


  0%|          | 0/630 [00:00<?, ?it/s]

  0%|          | 0/70 [00:00<?, ?it/s]

...

  0%|          | 0/70 [00:00<?, ?it/s]

Epoch 9: Validation Loss = 0.09629658527140106


  0%|          | 0/630 [00:00<?, ?it/s]

  0%|          | 0/70 [00:00<?, ?it/s]

Epoch 10: Validation Loss = 0.09216671303978988


In [25]:
torch.save(model.state_dict(), MODEL_PATH + 'model1')

In [27]:
the_model = torch.load(MODEL_PATH + 'model1')

In [ ]:
for params in the_model:
    print(params)

In [74]:
torch.cuda.empty_cache()

In [41]:
epochs = 10

for epoch in range(epochs):
    model2.train()
    for x1, x2, x3, y in tqdm(train_dataloader2, total = len(train_dataloader2), leave=False):
        x1, x2, x3, y = x1.to(device), x2.to(device), x3.to(device), y.to(device)
        x1 = x1.type(torch.cuda.FloatTensor)
        x2 = x2.type(torch.cuda.FloatTensor)
        x3 = x3.type(torch.cuda.FloatTensor)
        y = y.type(torch.cuda.FloatTensor)
        optimizer.zero_grad()
        pred = model2((x1, x2, x3))
        loss = criterion(pred, y)
        loss.backward()
        optimizer.step()
    model2.eval()
    val_loss = 0
    with torch.no_grad():
        for x1, x2, x3, y in tqdm(val_dataloader2, total = len(val_dataloader2), leave = False):
            x1, x2, x3, y = x1.to(device), x2.to(device), x3.to(device), y.to(device)
            x1 = x1.type(torch.cuda.FloatTensor)
            x2 = x2.type(torch.cuda.FloatTensor)
            x3 = x3.type(torch.cuda.FloatTensor)
            y = y.type(torch.cuda.FloatTensor)
            pred = model2((x1, x2, x3))
            loss = criterion(pred, y)
            val_loss += loss.item()
    val_loss /= (len(val_dataloader2)*batch_size)
    print(f'Epoch {epoch+1}: Validation Loss = {val_loss}')

  0%|          | 0/630 [00:00<?, ?it/s]

  0%|          | 0/70 [00:00<?, ?it/s]

Epoch 1: Validation Loss = 0.135145779805524


  0%|          | 0/630 [00:00<?, ?it/s]

  0%|          | 0/70 [00:00<?, ?it/s]

...

  0%|          | 0/70 [00:00<?, ?it/s]

Epoch 9: Validation Loss = 0.0630223360178726


  0%|          | 0/630 [00:00<?, ?it/s]

  0%|          | 0/70 [00:00<?, ?it/s]

Epoch 10: Validation Loss = 0.06640488284506968


In [42]:
torch.save(model2.state_dict(), MODEL_PATH + 'model2')

In [3]:
num_epochs = 200
learning_rate = 0.01

input_size = 1
hidden_size = 2
num_layers = 1

num_classes = 10

lstm = LSTMSurge(num_classes, input_size, hidden_size, num_layers)

criterion = torch.nn.MSELoss()
optimizer = torch.optim.Adam(lstm.parameters(), lr=learning_rate)

In [ ]:
for epoch in range(num_epochs):
    outputs = lstm(trainX)
    optimizer.zero_grad()
    
    # obtain the loss function
    loss = criterion(outputs, trainY)
    
    loss.backward()
    
    optimizer.step()
    if epoch % 100 == 0:
      print("Epoch: %d, loss: %1.5f" % (epoch, loss.item()))